In [5]:
import krakenex
from datetime import datetime, timedelta

In [7]:
kraken = krakenex.API()
last_hour = datetime.now() - timedelta(minutes = 10)
history = kraken.query_public(
    'OHLC', {'pair': 'ETHEUR', 'interval': 5, 'since': last_hour.timestamp(),'ascending': True})
print(history['result'])

{'XETHZEUR': [[1635377400, '3411.59', '3422.89', '3411.59', '3416.89', '3419.30', '5.79972406', 27], [1635377700, '3416.89', '3416.89', '3404.62', '3405.93', '3407.50', '43.16981168', 54], [1635378000, '3405.29', '3409.81', '3404.00', '3406.09', '3405.93', '22.55807706', 50], [1635378300, '3406.02', '3409.36', '3376.28', '3397.45', '3391.95', '507.88738425', 564], [1635378600, '3397.32', '3409.64', '3386.04', '3409.62', '3388.65', '59.82406984', 78], [1635378900, '3409.62', '3410.34', '3385.39', '3385.39', '3393.80', '7.12209344', 41], [1635379200, '3384.48', '3398.01', '3384.48', '3398.01', '3392.77', '24.16205239', 41], [1635379500, '3398.02', '3408.66', '3393.66', '3393.66', '3398.35', '56.55019762', 52], [1635379800, '3392.42', '3415.40', '3392.42', '3415.40', '3410.04', '7.89593799', 40], [1635380100, '3416.63', '3427.59', '3416.63', '3423.82', '3423.36', '16.09562668', 50], [1635380400, '3422.26', '3425.27', '3417.16', '3417.16', '3423.28', '40.64191481', 33], [1635380700, '3417.